In [29]:
dt["trial_1"]

<xarray.DataTree 'trial_1'>
Group: /trial_1
    Dimensions:                     (time: 1908, space: 3, keypoints: 3,
                                     individuals: 1, s3d_dims: 1024, RGB: 3,
                                     events: 4)
    Coordinates:
      * time                        (time) float64 15kB 0.0 0.005 ... 9.53 9.535
      * space                       (space) <U1 12B 'x' 'y' 'z'
      * keypoints                   (keypoints) <U8 96B 'beakTip' ... 'pellet'
      * individuals                 (individuals) <U6 24B 'Freddy'
    Dimensions without coordinates: s3d_dims, RGB, events
    Data variables: (12/24)
        position                    (time, space, keypoints, individuals) float64 137kB ...
        confidence                  (time, keypoints, individuals) float32 23kB ...
        velocity                    (time, space, keypoints, individuals) float64 137kB ...
        speed                       (time, keypoints, individuals) float64 46kB ...
        acceleration                (time, space, keypoints, individuals) float64 137kB ...
        pellet_beakTip_dist         (time) float64 15kB ...
        ...                          ...
        speed_troughs               (keypoints, individuals, time) int8 6kB ...
        speed_turning_points        (keypoints, individuals, time) int8 6kB ...
        angles                      (keypoints, individuals, time) float64 46kB ...
        angle_rgb                   (keypoints, individuals, time, RGB) float64 137kB ...
        labels                      (time, individuals) float64 15kB ...
        boundary_events             (events) float64 32B ...
    Attributes: (12/17)
        source_software:  DeepLabCut
        ds_type:          poses
        fps:              200.0
        time_unit:        seconds
        source_file:      D:/Alice/AK_data/rawdata/sub-03_id-Freddy/ses-000_date-...
        trial:            1
        ...               ...
        cam2_file:        2025-05-27_001_Freddy-cam-2.mp4
        dlc1_file:        2025-05-27_001_Freddy-cam-1DLC_resnet50_Felix_cross_Ses...
        dlc2_file:        2025-05-27_001_Freddy-cam-2DLC_resnet50_Felix_cross_Ses...
        poscat:           1
        num_pellets:      1
        human_verified:   1

In [35]:
speed_aux.shape

(286021, 3)

In [72]:
path1 = r"C:\Users\FM\Desktop\trainFreddy\labels\Trial_data2601_predictions_encoder_20251024_105616.nc"
path2 = r"C:\Users\FM\Desktop\trainFreddy\labels\Trial_data2601_predictions_decoder1_20251024_105616.nc"
path3 = r"C:\Users\FM\Desktop\trainFreddy\labels\Trial_data2601_predictions_decoder2_20251024_105616.nc"
path4 = r"C:\Users\FM\Desktop\trainFreddy\labels\Trial_data2601_predictions_decoder3_20251024_105616.nc"
path5 = r"C:\Users\FM\Desktop\trainFreddy\labels\Trial_data2601_predictions_corr_20251024_105616.nc"

encoder = TrialTree.load(path1)
decoder1 = TrialTree.load(path2)
decoder2 = TrialTree.load(path3)
decoder3 = TrialTree.load(path4)
corr = TrialTree.load(path5)

In [60]:
def get_time_labels(dt, trial_num):
    time = dt.sel(trials=trial_num).time.values
    labels = dt.sel(trials=trial_num).labels.squeeze().values
    return time, labels

from moveseg.utils.labels import load_motif_mapping
motif_mappings = load_motif_mapping(r"C:\Users\FM\Documents\Akseli\Code\MovFormer\configs_global\mapping.txt")

In [ ]:
from moveseg.utils.labels import plot_motif_segments
import matplotlib.pyplot as plt
trial = 20
time, labels = get_time_labels(encoder, trial)
fig, ax = plt.subplots(figsize=(15, 1))
plot_motif_segments(ax, time, labels, motif_mappings)


time, labels = get_time_labels(decoder1, trial)
fig, ax = plt.subplots(figsize=(15, 1))
plot_motif_segments(ax, time, labels, motif_mappings)


time, labels = get_time_labels(decoder2, trial)
fig, ax = plt.subplots(figsize=(15, 1))
plot_motif_segments(ax, time, labels, motif_mappings)


time, labels = get_time_labels(decoder3, trial)
fig, ax = plt.subplots(figsize=(15, 1))
plot_motif_segments(ax, time, labels, motif_mappings)

time, labels = get_time_labels(corr, trial)
fig, ax = plt.subplots(figsize=(15, 1))
plot_motif_segments(ax, time, labels, motif_mappings)

In [ ]:
path2 = r"C:\Users\FM\Desktop\trainFreddy\Trial_data2701.nc"
from moveseg.utils.io import TrialTree
import numpy as np
import os

dt = TrialTree.load(path2)
path = r"C:\Users\FM\Desktop\aux_data"

# Add auxiliary speed data to each trial
for trial in dt.trials:
    aux_filename = f"Freddy_20250527_01_trial{trial:03d}_v_aux_speed.npy"
    aux_file = os.path.join(path, aux_filename)
    
    if os.path.exists(aux_file):
        speed_aux = np.load(aux_file)
        speed_aux = speed_aux.squeeze()
        time_aux = np.arange(len(speed_aux)) / 30000  # Assuming 30 kHz sampling rate
        
        speed_aux_interp = np.interp(dt.sel(trials=trial).time.values, time_aux, speed_aux)
        
        # Access the DataTree node directly
        trial_node = dt[f"trial_{trial}"]
        ds = trial_node.ds.copy()
        ds['speed_aux'] = (('time',), speed_aux_interp)
        ds["speed_aux"].attrs['type'] = 'features' 
        trial_node.ds = ds
    else:
        print(f"No auxiliary file found for trial {trial}: {aux_file}")

# Verify it worked
speed_aux = dt.sel(trials=1)['speed_aux'].values

dt.to_netcdf(r"C:\Users\FM\Desktop\trainFreddy\Trial_data2701_with_aux_speed.nc")